In [1]:
import glob
import os
import shutil
from tqdm import tqdm
import pandas as pd

In [2]:
base_path = "/run/user/715136/gvfs/smb-share:server=wfs-biologie.top.gwdg.de,share=ubps-all$/Language_Acquisition/HIWI/HIWIstudies/OLDstudiesHIWI/SmallFaces in the Wild HIWI/Data/"
neon_scene_views_paths = glob.glob(os.path.join(base_path,"Neon/*/4*/*/*Scene*.mp4"))
pupilcore_scene_views_paths = glob.glob(os.path.join(base_path,"Pupil Core/raspi-*/*/*/world.mp4"))

In [68]:

neon_scene_views_paths = [path for path in neon_scene_views_paths if "ignore" not in path]
neon_scene_views_paths = [path for path in neon_scene_views_paths if "invalid" not in path]
neon_scene_view_nrs = [int(path.split("/")[-3]) for path in neon_scene_views_paths]


pupilcore_scene_views_paths = [path for path in pupilcore_scene_views_paths if "ignore" not in path]
pupilcore_scene_views_paths = [path for path in pupilcore_scene_views_paths if "invalid" not in path]
pupilcore_scene_view_nrs = [int(path.split("/")[-2].split("_")[0]) for path in pupilcore_scene_views_paths]

scene_view_paths = neon_scene_views_paths + pupilcore_scene_views_paths
scene_view_nrs = neon_scene_view_nrs + pupilcore_scene_view_nrs
scene_view_df = pd.DataFrame({"path":scene_view_paths,"scene_view_nr":scene_view_nrs})
scene_view_df["neon"] = scene_view_df["path"].str.contains("/Neon/")

scene_view_df["nr"] = range(len(scene_view_df))

scene_view_df["new_name"] = scene_view_df["scene_view_nr"].astype(str) + "_help_" +  scene_view_df["nr"].astype(str)

scene_view_df["gaze_position_path"] = None
scene_view_df["world_timestamps_path"] = None


scene_view_df_neon = scene_view_df[scene_view_df["neon"]]
scene_view_df_pupilcore = scene_view_df[~scene_view_df["neon"]]

In [ ]:
# # copies over all the video files that takes a while therefore it is commented out as default
# for index, row in tqdm(scene_view_df.iterrows()):
#     print(row["path"],row[["new_name"]])
#     shutil.copy(row['path'], f"/usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/{row['new_name']}.mp4")

In [ ]:
# processing this for the neon data
for index, row in scene_view_df_neon.iterrows():
    directory_path = os.path.dirname(row["path"])
    gaze_files = glob.glob(os.path.join(directory_path, "neon_player/exports/*/gaze_positions.csv"))
    gaze_path = gaze_files[0] if gaze_files else None
    scene_view_df_neon.at[index, "gaze_position_path"] = gaze_path

    folder_annotation_path = f"/usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/eye_tracking_annotations/{row['new_name']}"
    print(folder_annotation_path)
    os.makedirs(folder_annotation_path, exist_ok=True)
    
    if gaze_path is not None:
        shutil.copy(gaze_path, os.path.join(folder_annotation_path, "gaze_positions.csv"))
    world_timestamps_files = glob.glob(os.path.join(directory_path, "neon_player/exports/*/world_timestamps.csv"))
    world_timestamps_path = world_timestamps_files[0] if world_timestamps_files else None
    scene_view_df_neon.at[index, "world_timestamps_path"] = world_timestamps_path
    if world_timestamps_path is not None:
        shutil.copy(world_timestamps_path, os.path.join(folder_annotation_path, "world_timestamps.csv"))


In [ ]:
# processing this for the pupilcore data
example_path = os.path.dirname(scene_view_df_pupilcore["path"][50])
glob.glob(os.path.join(example_path, "exports/*/gaze_positions.csv"))


for index, row in scene_view_df_pupilcore.iterrows():
    directory_path = os.path.dirname(row["path"])
    gaze_files = glob.glob(os.path.join(directory_path, "exports/*/gaze_positions.csv"))
    gaze_path = gaze_files[0] if gaze_files else None
    scene_view_df_pupilcore.at[index, "gaze_position_path"] = gaze_path

    folder_annotation_path = f"/usr/users/vhassle/datasets/Wortschatzinsel/head_mounted_data/eye_tracking_annotations/{row['new_name']}"
    print(folder_annotation_path)
    os.makedirs(folder_annotation_path, exist_ok=True)
    
    if gaze_path is not None:
        shutil.copy(gaze_path, os.path.join(folder_annotation_path, "gaze_positions.csv"))
        # timesteps is not reuired in this case

In [ ]:
# exports/*/gaze_positions.csv